In [89]:
!jupyter nbconvert --to script DistributedDataParallel_script.ipynb
#!python DistributedDataParallel_script.py --issingle
!rm -rf MNIST_data

[NbConvertApp] Converting notebook DistributedDataParallel_script.ipynb to script
[NbConvertApp] Writing 11101 bytes to DistributedDataParallel_script.py


## Initialize Azure ML

In [86]:
from azureml.core import Workspace, Experiment, Run
ws = Workspace(subscription_id='<YOUR SUBSCRIPTION ID>',
              resource_group='<Resource grp>',
              workspace_name='<Workspace name>')

experiment = Experiment(ws, 'MNIST-singlegpu')

from azureml.core import ComputeTarget
compute_target = ComputeTarget(ws, 'nc24sv3')

### Single GPU

In [68]:
from azureml.train.dnn import PyTorch
estimator = PyTorch(source_directory = '.',
                    pip_packages = ['tqdm'],
                    compute_target = compute_target,
                    entry_script = 'DistributedDataParallel_script.py',
                    script_params = {
                       '--issingle':""
                    },
                    use_gpu=True,
                   )

run = experiment.submit(estimator)

WARNING - framework_version is not specified, defaulting to version 1.1.


In [69]:
from azureml.widgets import RunDetails
RunDetails(run).show()

_UserRunWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', '…

### Distributed data parallel AML job - Not working
2 nodes 4 GPUS
Follow this
https://github.com/microsoft/AzureML-BERT/blob/master/pretrain/PyTorch/azureml_adapter.py

In [90]:
from azureml.train.dnn import PyTorch
from azureml.core.runconfig import MpiConfiguration

mpi = MpiConfiguration()
mpi.process_count_per_node = 4

estimator_distrib = PyTorch(source_directory = '.',
                    pip_packages = ['tqdm'],
                   compute_target = compute_target,
                   entry_script = 'DistributedDataParallel_script.py',
                   script_params = {'--isaml':"", '--backend' : 'nccl'},
                   use_gpu=True,
                   distributed_training = mpi,
                    node_count = 2
                   )


WARNING - framework_version is not specified, defaulting to version 1.1.


In [91]:
run_distrib = experiment.submit(estimator_distrib)
RunDetails(run_distrib).show()

_UserRunWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', '…